In [1]:
import os
import geoutils as gu

#from helpers import getV, getMH, make_affine_matrix, yield_map
from helpers import getV, getMH, make_affine_matrix, yield_map, gen_heightmap, thicken, split_layer, tesselate_layer ,gen_fill, package_commands

# Parameters & Settings

In [2]:
# Loading rasters

fns = [fn for fn in os.listdir("data") if fn.endswith(".tif")]
rasters = sorted([gu.Raster(os.path.join("data", fn)) for fn in fns], key=lambda r: r.bounds[:2])

# Defining location of 2 points with precisely known longitudes and latitudes in both settings
x0, z0 =  -170, -196
x1, z1 = -1144,  284

w0, h0 = 2533405,   1152605.5
w1, h1 = 2532702.5, 1152259.5

# Creating XZ transformation matrices (both ways)

mX, hX = getMH(x0, w0, x1, w1)
mZ, hZ = getMH(z0, h0, z1, h1)
m2c = make_affine_matrix([mX, mZ], [hX, hZ])

mX, hX = getMH(w0, x0, w1, x1)
mZ, hZ = getMH(h0, z0, h1, z1)
c2m = make_affine_matrix([mX, mZ], [hX, hZ])

# Getting real-world altitude of 2 points with precisely known altitudes in both settings
lonField, latField = 2534190, 1152375
lonEspla, latEspla = 2533030, 1152475
lonSorge, latSorge = 2533664, 1152563
lonTalus, latTalus = 2534600, 1153270

v0 = getV(rasters, lonField, latField)
v1 = getV(rasters, lonTalus, latTalus)

y0, y1 = 34.8, 66.8

# Creating world-to-Minecraft Y transformation matrix

mY, hY = getMH(v0, y0, v1, y1)
mY, hY = 1.386821367789237, -488.2573682347112
v2y = make_affine_matrix([mY], [hY])

# Defining command text

prefix = "summon falling_block ~ ~1 ~ {Time:1,BlockState:{Name:redstone_block},Passengers:[\
{id:armor_stand,Health:0,Passengers:[\
{id:falling_block,Time:1,BlockState:{Name:activator_rail},Passengers:[\
{id:command_block_minecart,Command:'gamerule commandBlockOutput false'},\
{id:command_block_minecart,Command:'data merge block ~ ~-2 ~ {auto:0}'},"

template = "{{id:command_block_minecart,Command:'{0}'}}"

fill_command = "fill {0} {1} {2} {3} {4} {5} {6} {7} {8}"

clone_command = "clone {0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10}"

suffix = ",{id:command_block_minecart,Command:'setblock ~ ~1 ~ command_block{auto:1,Command:\"fill ~ ~ ~ ~ ~-2 ~ air\"}'},\
{id:command_block_minecart,Command:'kill @e[type=command_block_minecart,distance=..1]'}]}]}]}"

strings = {
    "prefix":        prefix,
    "template":      template,
    "fill_command":  fill_command,
    "clone_command": clone_command,
    "suffix":        suffix
}

# Command Generation

In [26]:
x, z = -1465, -1211
xOut = (x + 64) // 128
zOut = (z + 64) // 128
xOut, zOut

(-11, -9)

In [ ]:
xMap, zMap = -2, -9
yield_map(rasters, strings, m2c, c2m, v2y, xMap, zMap)

# ML Optimization

In [ ]:
import random

import pyperclip as pc

from scipy.optimize import curve_fit
from IPython.display import clear_output

In [ ]:
left, top = -1465, -1211
right, bottom = 1728, 960

In [ ]:
def random_tp(xz_list, left, right, top, bottom):
    tp_command = "/tp MCRaisin {0} 70 {1}"
    
    try:
        while True:
            x = random.randrange(left, right)
            z = random.randrange(top, bottom)
            
            xz_list.append((x, z))
            
            cmd = tp_command.format(x, z)
            pc.copy(cmd)
            print(cmd)
            
            input("Press Enter to store the next command in your clipboard...")
            clear_output(wait=True)
    except KeyboardInterrupt:
        pass

In [ ]:
plt.scatter(src, tgt)

In [ ]:
def get_alts(xz_list, y_list, m2c, rasters):
    lonlat_list = convert_points(np.array(xz_list), m2c).tolist()
    ret = []
    
    for lon, lat in lonlat_list:
        v = getV(rasters, lon, lat)
        ret.append(v)
        
    src_list = np.array(ret)
    tgt_list = np.array(y_list)
    
    src_list = src_list[tgt_list != None]
    tgt_list = tgt_list[tgt_list != None]
    
    return src_list, tgt_list
        
src, tgt = get_alts(xz_list, y_list, m2c, rasters)

In [ ]:
# Finds a raster that matches a longitude and a latitude
def find_raster(rasters, lon, lat):
    for r in rasters:
        if sum(fits_bb(r, lon, lat)) == 2:
            return r
        
    return None

# Returns the altitude of a point defined by its latitude and longitude
def getV(rasters, lon, lat):
    r = find_raster(rasters, lon, lat)
    
    if r is None:
        return None
    
    return r.value_at_coords(lon, lat)

In [ ]:
# Transforms a whole set of points with a given matrix
def convert_points(points, mat):
    points = np.c_[ points, np.ones(points.shape[0])]
    return np.matmul(mat, points.T)[:2].T

In [ ]:
points = np.array(xz_list)
points.shape[0]

In [ ]:
xz_list = [
    (1043, -1099), (1240, -512),  (1416, -834),   (1437, -391),  (-100, 863),
    (1412, -1118), (-1086, -195), (-78, -331),    (-1020, -107), (-1413, 927),
    (141, -762),   (1132, 762),   (-520, -934),   (-1424, 926),  (-1262, 468),
    (1485, -535),  (1070, -919),  (-1031, -565),  (-1377, 950),  (-1253, -934),
    (-1219, -710), (144, 892),    (-726, -1182),  (-1035, 463),  (-1338, 341),
    (-1104, -592), (-206, -1107), (1243, -13),    (-480, -252),  (792, -799),
    (548, 542),    (-41, 27),     (995, -72),     (-581, -1176), (253, 302),
    (1685, -655),  (1506, -291),  (-1326, -1137), (-131, -1194), (1232, -1014),
    (383, -135),   (1226, -1068), (-72, -1037),   (506, 547),    (1226, -652),
    (-201, -874),  (-23, 352),    (1548, -314),   (1364, -263),  (-894, 345),
    (-792, 5),     (924, -315),   (1395, -419),   (1019, 145),   (156, 620),
    (969, 897),    (-980, -858),  (-484, -741)
]

xz_list

In [ ]:
y_list = [
    58, None, 55, None, 37,
    66, 65, 51, None, 61,
    60, None, 62, 61, 69,
    46, 56, 69, 60, 94,
    104, 32, 68, None, 69,
    84, 64, 31, None, 52,
    29, None, None, 62, 49,
    53, 44, 85, 65, 56,
    None, 57, 63, None, None,
    61, 57, 46, None, None,
    None, 44, None, None, 41,
    None, 83, 53
]

len(y_list)

In [ ]:
print(len(xz_list))

In [ ]:
xz_list = []

In [ ]:
random_tp(xz_list, left, right, top, bottom)

In [ ]:
coeffs, _ = curve_fit(objective, src, tgt)
m, h = coeffs

In [ ]:
m, h

In [ ]:
v2y = make_affine_matrix([m], [h])
v2y

In [ ]:
x = random.randrange(left, right)
z = random.randrange(top, bottom)

In [ ]:
def objective(x, m, h):
    return m * x + h

# New features

In [ ]:
def load_ranges():
    try:
        with open("ranges.json", "r") as f:
            ranges = json.load(f)
    except FileNotFoundError:
        ranges = dict()
        
    return ranges

def reset_ranges():
    ranges = dict()
    
    with open("ranges.json", "w") as f:
        json.dump(ranges, f)
    
def save_range(pos, cmds, Y):
    ranges = load_ranges()
    x, z = pos
    x, z = str(x), str(z)
    y0, y1 = Y
    
    if ranges.get(x) is None:
        ranges[x] = dict()
    
    ranges[x][z] = {
        "start": int(y0),
        "end": int(y1),
        "cmds": cmds
    }
    
    with open("ranges.json", "w") as f:
        json.dump(ranges, f)

def mapTopLeft(mX, mZ):
    return mX * 128 - 64, mZ * 128 - 64

def package_commands(cmds, strings):
    max_chars = 32500
    prefix = strings["prefix"]
    suffix = strings["suffix"]
    
    def_len = len(prefix) + len(suffix)
    avail_space = max_chars - def_len
    lens = [len(cmd) + 1 for cmd in cmds]
    
    idx = 0
    batches = []
    
    while lens:
        idx = (np.cumsum(lens) <= avail_space).argmin(0)
        
        if idx == 0:
            idx = len(cmds)
            
        string = prefix + ",".join(cmds[:idx]) + suffix
        batches.append(string)
        lens = lens[idx:]
        cmds = cmds[idx:]
        
    for i, b in enumerate(batches):
        input("Press Enter to store the next command in your clipboard...")
        pc.copy(b)
        print(f"Batch {i + 1} of {len(batches)}")
        
def wipe_area(strings, pos, blocks, Y=None):
    x, z = pos
    
    block_map = {
        "d": "diamond_block",
        "g": "gold_block"
    }
    
    to_wipe = [v for k, v in block_map.items() if k in blocks]
        
    if Y is None:
        print("A")
        try:
            ranges = load_ranges()
            y0, y1 = tuple(ranges[str(x)][str(z)].values())[:2]
        except KeyError:
            print("The area (" + str(x) + ", " + str(z) + ") has not been seen yet.")
            return
    else:
        print("B")
        y0, y1 = Y

    x0, z0 = mapTopLeft(x, z)
    cmds = [gen_fill(strings, (127, 127, x0, z0), Y=y, block0="air", mode="replace", block1=block) for y in range(y0, y1) for block in to_wipe]
    package_commands(cmds, strings)

In [ ]:
# Turns a Minecraft heightmap into a set of individual commands yielded as batches
def cubify(arr, strings, shift=(0, 0)):
    multilayer, Y0, depth = thicken(arr.T)
    xShift, zShift = shift
    cmds = []
    
    for y in range(depth):
        layer = multilayer[..., y]
        sublayers = split_layer(layer)
        edges = []
        
        for (sub_layer, i, j) in sublayers:
            edges += tesselate_layer(sub_layer, i, j, xShift, zShift)
            
        cmds += [gen_fill(strings, elem, Y0 + y, "diamond_block", mode="keep") for elem in edges]
    
    package_commands(cmds, strings)
    
    return cmds, (Y0, Y0 + depth)

In [ ]:
def yield_map(rasters, strings, m2c, c2m, v2y, xMap, zMap, overwrite=False):
    xS, zS = mapTopLeft(xMap, zMap)
    arr = gen_heightmap(rasters, m2c, c2m, v2y, xS, zS)
    cmds, Y = cubify(arr, strings, shift=(xS, zS))
    save_range((xMap, zMap), cmds, Y)

In [ ]:
from helpers import gen_fill
import pyperclip as pc
import numpy as np
import json
from matplotlib import pyplot as plt

In [ ]:
wipe_area(strings, (1, -2), "dg")

In [ ]:
ranges = {
    np.int32(4) : 2
}
    
with open("temp.json", "w") as f:
    json.dump(ranges, f)

In [ ]:
np.int32(4)

In [ ]:
x0, z0 = mapTopLeft(1, -2)
y0, y1 = 24, 70
cmds = [gen_fill(strings, (127, 127, x0, z0), Y=y, block0="air", mode="replace", block1="diamond_block") for y in range(y0, y1)]
package_commands(cmds, strings)